Горностаев Александр, ML-11

In [14]:
# at first, mount the drive

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [217]:
# сразу импортирую что нужно
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from scipy.stats import rv_discrete

from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.compose import make_column_transformer
import numpy as np
from sklearn.decomposition import PCA

In [218]:
# рабочий, но не использовал по итогу
'''
imputes values from distribution for each given column
target missing values are np.nan
'''
class DiscreteDistributionImputer():
  def __init__(self, random_state=42):
    # contains distributions to generate random stuff from
    self.distributions = []
    # to fix randomness for result recheck
    self.random_state = random_state

  def fit(self, X, y=None):
    columns = []

    if isinstance(X, pd.DataFrame):
      columns = X.values.T
    elif isinstance(X, np.ndarray):
      columns = X.T
    else:
      print("given X is neither DataFrame nor ndarray")

    # clear distributions
    self.distributions = []
    for col in columns:
      # must consider only not nans
      notnans_mask = ~np.isnan(col)
      # must create distribution for each column now
      unique_vals, counts = np.unique(col[notnans_mask], return_counts=True)
      probabilities = counts / counts.sum()
      distribution = rv_discrete(values=(unique_vals, probabilities))
      self.distributions.append(distribution)
    
    return self

  def fit_transform(self, X, y = None):
    # fit first to create distributions
    self.fit(X, y)

    # transform
    return self.transform(X)
    

  def transform(self, X):
    columns = []

    # cast to float because only floats support np.nan
    if isinstance(X, pd.DataFrame):
      columns = X.values.T.copy().astype(np.float)
    elif isinstance(X, np.ndarray):
      columns = X.T.copy().astype(np.float)
    else:
      print("given X is neither DataFrame nor ndarray")

    for i, col in enumerate(columns):
      distribution = self.distributions[i]
      nan_cells_mask = np.isnan(col)
      # careful here, where returns a one sized tuple (<value>, )
      nan_cells_indices, = np.where(nan_cells_mask)
      random_values = distribution.rvs(size=nan_cells_indices.size)
      for index, random in zip(nan_cells_indices, random_values):
        col[index] = random

    return columns.T

  def get_params(self, deep=True):
    return {}

  def set_params(self, **kwargs):
    return self

In [96]:
'''
impute nan using KNN algorithm but slightly modified
By default KNNImputer will calculate means (or medians), this implementation will instead look for closest 
values (for discrete features)
'''
from sklearn.impute import KNNImputer
class KNNWithDiscreteImputer(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns_with_nans=[]):
        print(columns_with_nans)
        self.columns_with_nans = columns_with_nans
    
    def fit(self, X, y=None):
        return self
    
    '''
    finds closest value to one of the given unique and sets to each cell
    '''
    @staticmethod
    def set_closest_value(matrix, column, unique_values, indices):
      for i in indices:
        distance = []
        for j in range(len(unique_values)):
          distance.append(abs(matrix[i,column]-unique_values[j]))
        
        min_distance = min(distance)
        min_idx = distance.index(min_distance)
        matrix[i, column] = unique_values[min_idx]
    
    def transform(self, X):
        if (len(self.columns_with_nans) == 0):
            print("no target columns were given")
            return X
        
        unique_target_cols_values = {}
        
        for i in self.columns_with_nans:
            # indeces where nans in this column present
            nans_idx = np.array(X.index[X[i].isnull()].tolist())

            unique_col_values = X[i].unique()
            # remove nans
            unique_col_values = unique_col_values[~np.isnan(unique_col_values)]
            col_index = X.columns.tolist().index(i)
            
            unique_target_cols_values[i] = (unique_col_values, col_index, nans_idx)
            
        imputer = KNNImputer(n_neighbors=3, weights='uniform', metric='nan_euclidean', copy=True) # число соседей подобрал эмпирически
        imputed_data = imputer.fit_transform(X.values)
        
        for col_name in self.columns_with_nans:
            unique_values, col_index, nans_idx = unique_target_cols_values[col_name]
            self.set_closest_value(imputed_data, col_index, unique_values, nans_idx)

        return imputed_data

In [219]:
# для перевода дат в месяца
'''
finds a min date per Series (of type pd.Timestamp)
then subtracts it from each date
the result per each column is a column with amount of days
'''
class DatePreprocessor():
  def __init__(self, random_state=42):
    self.min_dates = []

  def fit(self, X, y=None):
    # columns = []
    # if (len(X.shape) == 1):
    #   print("1d array is not supported, expected 2d")

    # if isinstance(X, pd.DataFrame):
    #   columns = X.values.T
    # elif isinstance(X, np.ndarray):
    #   columns = X.T
    # else:
    #   print("given X is neither DataFrame nor ndarray")

    # for i in columns:
    #   min_date, = X.min()
    #   if isinstance(min_date, pd.Timestamp):
    #     self.min_dates.append(min_date)
    #   else:
    #     print(type(min_date))
    #     print("passed column is not of np.datetime64")

    return self

  def fit_transform(self, X, y = None):
    # fit first to create distributions
    self.fit(X, y)

    # transform
    return self.transform(X)
    

  def transform(self, X):
    columns = []
    if (len(X.shape) == 1):
      print("1d array is not supported, expected 2d")

    if isinstance(X, pd.DataFrame):
      columns = X.values.T
    elif isinstance(X, np.ndarray):
      columns = X.T
    else:
      print("given X is neither DataFrame nor ndarray")

    print(columns.dtype)
    dates = np.ones(X.T.shape, dtype=np.int)

    for index, col in enumerate(columns):
      dates[index] = np.array(list(map(lambda date: pd.Timestamp(date).month, col)))

    return dates.T

  def get_params(self, deep=True):
    return {}

  def set_params(self, **kwargs):
    return self

In [220]:
# для каста определенных столбцов к интам
'''
casts all passed columns to int

may raise error if cannot cast series to int (if, e.g., nan is encountered)
'''
class CategoricalFeaturesToIntPreprocessor(BaseEstimator, TransformerMixin):
  def __init__(self):
      pass
  
  def fit(self, X, y=None):
      return self
  
  def fit_transform(self, X, y = None):
    # fit first to create distributions
    self.fit(X, y)

    # transform
    return self.transform(X)
  
  def transform(self, X):
    columns = []
    print(type(X.head()))
    print(X.head(), X.shape)
    columns = X.T

    for index in range(len(columns)):
      columns[index] = columns[index].astype(np.int)
    
    return columns.T

In [221]:
# для проверки модели с просто дропнутыми данными
'''
drops nans
'''
class NanDropper(BaseEstimator, TransformerMixin):
  def __init__(self):
      pass
  
  def fit(self, X, y=None):
      return self
  
  def fit_transform(self, X, y = None):
    # fit first to create distributions
    self.fit(X, y)

    # transform
    return self.transform(X)
  
  def transform(self, X, *y):
    X = X.dropna()
    return X

In [113]:
def pandarizer(columns, *placed_to_first, **options):
    if (len(placed_to_first) != 0):
        columns = names_at_start(columns, *placed_to_first)
        
    def framerize(x):
        frame = None
        if isinstance(x, pd.DataFrame):
            frame = x
        else:
            frame = pd.DataFrame(x, columns = columns)
        
        if len(options) > 0:
            int_columns = options['toint']
            for int_column in int_columns:
                frame[int_column] = frame[int_column].astype(np.int)
        return frame
            
    return FunctionTransformer(lambda x: framerize(x))

def names_at_start(columns, *placed_to_first):
    columns = columns.copy()
    if not isinstance(columns, list):
        columns = columns.tolist()
        
    for name in placed_to_first[::-1]:
        if name not in columns:
            print(name, "not in list")
        columns.remove(name)
        columns.insert(0, name)
    return columns

def without(columns, *without_names):
    columns = columns.copy()
    if not isinstance(columns, list):
        columns = columns.tolist()
        
    for name in without_names:
        columns.remove(name)
    return columns

## Подгрузить данные



In [222]:
gdrive_path_train = "/content/drive/MyDrive/Технопарк 2021 ML/lecture05/Train.csv"
local_path_train = "Train.csv"
gdrive_path_test = "/content/drive/MyDrive/Технопарк 2021 ML/lecture05/Test.csv"
local_path_test = "Test.csv"

data_train = pd.read_csv(local_path_train, index_col=None)
data_test = pd.read_csv(local_path_test, index_col=None)

# сразу дропаю айдишники
data_train['date'] = pd.to_datetime(data_train['date'])
data_test['date'] = pd.to_datetime(data_test['date'])
data_train.drop(data_train.columns[[0]], axis=1,inplace=True)
data_test.drop(data_test.columns[[0]], axis=1,inplace=True)

In [60]:
data_train.head()

,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,2011-01-01,385,NaN,1,59,3,0,25.0,NaN,2,...,0,0,0,0,0,0,0,0,0,2489000
1,2011-01-01,247,NaN,5,53,2,1,15.0,1.0,2,...,0,0,0,0,0,0,0,0,0,1995000
2,2011-01-01,115,0.0,7,61,3,0,10.0,1.0,1,...,0,0,0,0,0,0,0,0,0,3993000
3,2011-01-01,491,1.0,5,61,3,1,30.0,NaN,0,...,0,0,0,0,0,0,0,0,0,5709000
4,2011-01-01,623,NaN,3,51,2,1,30.0,NaN,1,...,0,0,0,0,0,0,0,0,0,4374000


## Поиск модели для предсказания стоимости квартиры
Здесь сразу дроблю трейн дата сет на трейн и тест - надо

In [62]:
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(data_train.iloc[:, :-1], data_train.iloc[:, -1], shuffle=False)

Относительная точность


In [223]:
def relative_accuracy(y_true, y_pred): 
  return 1 - (np.abs((y_true - y_pred) / y_train_test).mean())

In [93]:
data_train_transformed_with_date = np.hstack((np.ones((data_train.shape[0], 1)), data_train_transformed))

In [95]:
filled_data_train = pd.DataFrame(data_train_transformed_with_date, columns=data_train.columns)
filled_data_train['date'] = data_train['date']

,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,2011-01-01,385.0,1.0,1.0,59.0,3.0,0.0,25.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2489000.0
1,2011-01-01,247.0,1.0,5.0,53.0,2.0,1.0,15.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1995000.0
2,2011-01-01,115.0,0.0,7.0,61.0,3.0,0.0,10.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3993000.0
3,2011-01-01,491.0,1.0,5.0,61.0,3.0,1.0,30.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5709000.0
4,2011-01-01,623.0,0.0,3.0,51.0,2.0,1.0,30.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4374000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2012-03-01,655.0,0.0,2.0,44.0,2.0,0.0,20.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1838000.0
99996,2012-03-01,175.0,1.0,10.0,60.0,3.0,0.0,30.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6355000.0
99997,2012-03-01,199.0,1.0,4.0,64.0,2.0,1.0,30.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6695000.0
99998,2012-03-01,226.0,1.0,10.0,35.0,1.0,1.0,15.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2544000.0


In [97]:
filled_data_train.isna().any(axis=1).any()

False

In [98]:
# filled_data_train.to_csv("filled_data_train.csv", sep=',', index=False)

### CatBoost 

In [126]:
from catboost import Pool, CatBoostRegressor

In [ ]:
# preprocess_model = make_pipeline(
#     make_column_transformer(
#         # convert date to months
#         (DatePreprocessor(), ['date']),
#         # fill nans for columns using their distributions
#         (DiscreteDistributionImputer(), ['build_tech', 'g_lift']),
#         # fill left nans simply
#         remainder=SimpleImputer()
#     ),
#     # pandarize for other columntransform conversion, date, build_Tech and g_lift labels go first after first CT conversion
#     pandarizer(X_labels, 'date', 'build_tech', 'g_lift'),
#     # drop kw columns that are highly disbalanced
# #     make_column_transformer(
# #         ('drop', ["kw{}".format(i) for i in range(1, 14) if i != 2]),
# #         remainder='passthrough'
# #     ),
#     # pandarize again to check that everything is as must
# #     pandarizer(without(X_labels, *["kw{}".format(i) for i in range(1, 14) if i != 2]), 
# #                'date', 'build_tech', 'g_lift',
# #                toint=['date', 'build_tech', 'g_lift', 'street_id', 'floor', 'area', 'rooms', 'balcon', 'n_photos', 'kw2']
# #               ),
#     pandarizer(X_labels, 
#            'date', 'build_tech', 'g_lift',
#            toint=['date', 'build_tech', 'g_lift', 'street_id', 'floor', 'area', 'rooms', 'balcon', 'n_photos', 'kw2']
#           ),
#     'passthrough'
# )

In [100]:
make_column_transformer(
    (SimpleImputer(),)
).fit_transform(data_train)

ValueError: not enough values to unpack (expected 2, got 1)

Этот пайплайн должен:
1. Менять даты на месяца
2. Пропущенные значения выводить с помощью KNNImputer. Если они дискретные - то будет выбираться к дискретному значение.
3. Затем будет делать датафрейм, чтобы категориальные фичи преобразовать обратно в целочисленные
4. Кормить датасет нужной модели

Поскольку KNN долгий - то и работать оно будет долго

In [107]:
arr = [12, 3, 4, *[34, 54, 2]]

In [108]:
arr

[12, 3, 4, 34, 54, 2]

In [201]:
def create_preprocess_model_for_columns(columns):
    preprocess_model = make_pipeline(
        make_column_transformer(
            # convert date to months
            (DatePreprocessor(), ['date']),

            remainder= # fill all nans with KNN and for discrete values choose closest
            (KNNWithDiscreteImputer(columns_with_nans=['build_tech', 'g_lift']))
        ),
        # pandarize for other columntransform conversion, date label goes first after first step,
        # toint=[columns] columns will become ints in new pandas table after this step
        pandarizer(columns, 'date',
                   toint=['date', 'build_tech', 
                          'g_lift', 'floor',
                          'rooms', 'balcon',
                          'n_photos', 'street_id',
                         *["kw{}".format(i) for i in range(1, 14)]
                         ]),
        'passthrough'
    )
    return preprocess_model

In [121]:
preprocess_model = make_pipeline(
    make_column_transformer(
        # convert date to months
        (DatePreprocessor(), ['date']),

        remainder= # fill all nans with KNN and for discrete values choose closest
        (KNNWithDiscreteImputer(columns_with_nans=['build_tech', 'g_lift']))
    ),
    # pandarize for other columntransform conversion, date label goes first after first step,
    # toint=[columns] columns will become ints in new pandas table after this step
    pandarizer(data_train.columns, 'date',
               toint=['date', 'build_tech', 
                      'g_lift', 'floor',
                      'rooms', 'balcon',
                      'n_photos', 'street_id',
                     *["kw{}".format(i) for i in range(1, 14)]
                     ]),
    'passthrough'
)

['build_tech', 'g_lift']


In [122]:
%%time
data_train_preprocessed = preprocess_model.fit_transform(data_train)

datetime64[ns]
['build_tech', 'g_lift']
Wall time: 4min 20s


In [116]:
# integerizer = pandarizer(data_train_preprocessed.columns,
#                          toint=['date', 'build_tech', 
#                                 'g_lift', 'floor', 'rooms', 
#                                 'balcon','n_photos',
#                                 'street_id', 
#                                 *["kw{}".format(i) for i in range(1, 14)]])
# data_train_preprocessed = integerizer.transform(data_train_preprocessed)

In [123]:
data_train_preprocessed

,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,1,385,1,1,59.0,3,0,25.0,1,2,...,0,0,0,0,0,0,0,0,0,2489000.0
1,1,247,1,5,53.0,2,1,15.0,1,2,...,0,0,0,0,0,0,0,0,0,1995000.0
2,1,115,0,7,61.0,3,0,10.0,1,1,...,0,0,0,0,0,0,0,0,0,3993000.0
3,1,491,1,5,61.0,3,1,30.0,0,0,...,0,0,0,0,0,0,0,0,0,5709000.0
4,1,623,0,3,51.0,2,1,30.0,0,1,...,0,0,0,0,0,0,0,0,0,4374000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3,655,0,2,44.0,2,0,20.0,1,1,...,0,0,0,0,0,0,0,0,0,1838000.0
99996,3,175,1,10,60.0,3,0,30.0,0,5,...,0,0,0,0,0,0,0,0,0,6355000.0
99997,3,199,1,4,64.0,2,1,30.0,0,3,...,0,0,0,0,0,0,0,0,0,6695000.0
99998,3,226,1,10,35.0,1,1,15.0,1,1,...,0,0,0,0,0,0,0,0,0,2544000.0


In [124]:
data_train_preprocessed.to_csv("TrainPreprocessed.csv", sep=",", index=False)

In [179]:
model = CatBoostRegressor(learning_rate=0.1, eval_metric ='MAE', task_type='GPU', iterations=5000, verbose=50) 

In [180]:
categorical_features = ['date', 'build_tech', 'street_id']

In [170]:
pipeline_model = model
# make_pipeline(
# #     preprocess_model,
# #     model
# )

In [181]:
X_train_train, X_train_test, y_train_train, y_train_test\
= train_test_split(data_train_preprocessed.iloc[:, :-1], data_train_preprocessed.iloc[:, -1], shuffle=False,test_size=0.9)

In [182]:
train_set = Pool(data=X_train_train, label=y_train_train)
test_set = Pool(data=X_train_test, label=y_train_test)

In [184]:
pipeline_model.fit(train_set, eval_set=test_set)

0:	learn: 2289210.7776000	test: 2335116.4928000	best: 2335116.4928000 (0)	total: 16ms	remaining: 1m 20s
50:	learn: 2191723.1104000	test: 2238773.1114667	best: 2238773.1114667 (50)	total: 437ms	remaining: 42.4s
100:	learn: 2051418.9312000	test: 2105767.4353778	best: 2105767.4353778 (100)	total: 902ms	remaining: 43.7s
150:	learn: 1947486.0032000	test: 2003471.3827556	best: 2003454.4526222 (148)	total: 1.35s	remaining: 43.2s
200:	learn: 1926650.2656000	test: 1984709.8823111	best: 1984617.9498667 (193)	total: 1.77s	remaining: 42.2s
250:	learn: 1908620.9024000	test: 1967296.0568889	best: 1967296.0568889 (250)	total: 2.2s	remaining: 41.7s
300:	learn: 1888784.9984000	test: 1948033.9342222	best: 1948033.9342222 (300)	total: 2.65s	remaining: 41.4s
350:	learn: 1835917.9264000	test: 1893816.0014222	best: 1893816.0014222 (350)	total: 3.09s	remaining: 40.9s
400:	learn: 1802574.8480000	test: 1858227.2227556	best: 1858227.2227556 (400)	total: 3.55s	remaining: 40.7s
450:	learn: 1788010.7008000	test: 1

3800:	learn: 1500394.7008000	test: 1597599.6529778	best: 1597597.4684444 (3798)	total: 34.5s	remaining: 10.9s
3850:	learn: 1498997.8624000	test: 1596734.0316444	best: 1596734.0316444 (3850)	total: 35s	remaining: 10.4s
3900:	learn: 1495158.9888000	test: 1592982.0956444	best: 1592981.9136000 (3899)	total: 35.5s	remaining: 9.99s
3950:	learn: 1494052.8640000	test: 1592060.7687111	best: 1592053.1228444 (3927)	total: 36s	remaining: 9.55s
4000:	learn: 1493646.1312000	test: 1591896.7466667	best: 1591896.2005333 (3998)	total: 36.6s	remaining: 9.14s
4050:	learn: 1491890.1760000	test: 1590045.3546667	best: 1590045.3546667 (4050)	total: 37.1s	remaining: 8.69s
4100:	learn: 1491375.7184000	test: 1589760.8192000	best: 1589760.8192000 (4100)	total: 37.5s	remaining: 8.23s
4150:	learn: 1490318.9504000	test: 1589120.7509333	best: 1589072.5091556 (4144)	total: 38s	remaining: 7.78s
4200:	learn: 1489370.9312000	test: 1588054.8807111	best: 1588054.8807111 (4200)	total: 38.5s	remaining: 7.33s
4250:	learn: 148

GPU

In [212]:
cat_feat=["date","build_tech","street_id"]
model = CatBoostRegressor(loss_function='MAE',cat_features=cat_feat,early_stopping_rounds=22,eval_metric = 'MAE', task_type="GPU") # 22 выбрал рандомно
# как я понял из доки, кросс вал тут встроена в гридсерч
# беру 5 фолдов и размер трейна 0.8

grid = {'learning_rate': [0.1,0.5,1],
        'depth': [8,10,12],
        'iterations': [250,500,1000]}
grid_search_result = model.grid_search(grid, 
                                       X=data_train_preprocessed.iloc[:, :-1], 
                                       y=data_train_preprocessed.iloc[:, -1],
                                       verbose = False,
                                       partition_random_seed = 123,
                                       cv = 5,
                                       train_size=0.8,
                                       plot=False)
grid_search_result


0:	learn: 4805219.5328000	test: 4806987.3664000	best: 4806987.3664000 (0)	total: 32.6ms	remaining: 8.12s
1:	learn: 4805219.5328000	test: 4806987.3664000	best: 4806987.3664000 (0)	total: 57.4ms	remaining: 7.12s
2:	learn: 4805219.5328000	test: 4806987.3664000	best: 4806987.3664000 (0)	total: 80ms	remaining: 6.59s
3:	learn: 4805220.3520000	test: 4806986.9568000	best: 4806986.9568000 (3)	total: 112ms	remaining: 6.86s
4:	learn: 4805219.5328000	test: 4806987.3664000	best: 4806986.9568000 (3)	total: 143ms	remaining: 7.01s
5:	learn: 4805219.5328000	test: 4806987.3664000	best: 4806986.9568000 (3)	total: 167ms	remaining: 6.77s
6:	learn: 4805219.5328000	test: 4806986.9568000	best: 4806986.9568000 (3)	total: 180ms	remaining: 6.24s
7:	learn: 4805219.5328000	test: 4806987.3664000	best: 4806986.9568000 (3)	total: 234ms	remaining: 7.09s
8:	learn: 4805219.5328000	test: 4806987.3664000	best: 4806986.9568000 (3)	total: 263ms	remaining: 7.05s
9:	learn: 4805219.5328000	test: 4806986.9568000	best: 4806986.9

80:	learn: 4805215.0272000	test: 4806983.6800000	best: 4806982.8608000 (75)	total: 2.13s	remaining: 4.45s
81:	learn: 4805215.4368000	test: 4806982.8608000	best: 4806982.8608000 (75)	total: 2.16s	remaining: 4.43s
82:	learn: 4805215.0272000	test: 4806982.4512000	best: 4806982.4512000 (82)	total: 2.19s	remaining: 4.41s
83:	learn: 4805215.0272000	test: 4806982.4512000	best: 4806982.4512000 (82)	total: 2.21s	remaining: 4.37s
84:	learn: 4805215.0272000	test: 4806983.6800000	best: 4806982.4512000 (82)	total: 2.24s	remaining: 4.35s
85:	learn: 4805214.6176000	test: 4806982.4512000	best: 4806982.4512000 (82)	total: 2.26s	remaining: 4.31s
86:	learn: 4805214.6176000	test: 4806982.4512000	best: 4806982.4512000 (82)	total: 2.29s	remaining: 4.29s
87:	learn: 4805214.6176000	test: 4806982.8608000	best: 4806982.4512000 (82)	total: 2.31s	remaining: 4.25s
88:	learn: 4805214.6176000	test: 4806982.4512000	best: 4806982.4512000 (82)	total: 2.33s	remaining: 4.21s
89:	learn: 4805215.0272000	test: 4806982.86080

KeyboardInterrupt: 

CPU

In [213]:
from catboost import CatBoostRegressor
cat_feat=["date","build_tech","street_id", "kw2", "balcon", "g_lift", "floor"]
model2 = CatBoostRegressor(loss_function='MAE',cat_features=cat_feat,learning_rate=0.1,depth=8,eval_metric = 'MAE', iterations=1000) # 22 выбрал рандомно
# как я понял из доки, кросс вал тут встроена в гридсерч
# беру 5 фолдов и размер трейна 0.8

# grid = {'learning_rate': [0.1,1],
#         'depth': [8,10,12],
#         'iterations': [1000]}
# grid_search_result = model.grid_search(grid, 
#                                        X=data_train_preprocessed.iloc[:, :-1], 
#                                        y=data_train_preprocessed.iloc[:, -1],
#                                        verbose = False,
#                                        partition_random_seed = 123,
#                                        cv = 5,
#                                        train_size=0.8,
#                                        plot=False)
# grid_search_result
model2.fit(X_train_train, y_train_train)

0:	learn: 2164360.3283391	total: 26.2ms	remaining: 26.2s
1:	learn: 2057272.9078554	total: 52.1ms	remaining: 26s
2:	learn: 1951272.2864260	total: 76ms	remaining: 25.2s
3:	learn: 1851810.7237520	total: 100ms	remaining: 25s
4:	learn: 1787065.7820791	total: 124ms	remaining: 24.7s
5:	learn: 1721941.0199425	total: 149ms	remaining: 24.6s
6:	learn: 1672558.1675510	total: 173ms	remaining: 24.6s
7:	learn: 1638371.8562302	total: 189ms	remaining: 23.4s
8:	learn: 1583493.6573482	total: 212ms	remaining: 23.4s
9:	learn: 1544802.1301426	total: 239ms	remaining: 23.6s
10:	learn: 1497770.7416908	total: 265ms	remaining: 23.8s
11:	learn: 1456062.1284593	total: 291ms	remaining: 23.9s
12:	learn: 1422047.9878913	total: 314ms	remaining: 23.9s
13:	learn: 1396748.1602468	total: 346ms	remaining: 24.4s
14:	learn: 1371417.5309449	total: 371ms	remaining: 24.4s
15:	learn: 1350198.3369312	total: 397ms	remaining: 24.4s
16:	learn: 1327889.5264282	total: 422ms	remaining: 24.4s
17:	learn: 1313334.3092649	total: 450ms	rema

152:	learn: 901652.6291958	total: 4.19s	remaining: 23.2s
153:	learn: 901000.1642045	total: 4.22s	remaining: 23.2s
154:	learn: 899251.7002374	total: 4.24s	remaining: 23.1s
155:	learn: 897706.3493715	total: 4.27s	remaining: 23.1s
156:	learn: 896111.4809632	total: 4.3s	remaining: 23.1s
157:	learn: 892675.2757458	total: 4.32s	remaining: 23s
158:	learn: 891956.0395995	total: 4.35s	remaining: 23s
159:	learn: 889483.8501424	total: 4.38s	remaining: 23s
160:	learn: 888138.5808850	total: 4.41s	remaining: 23s
161:	learn: 887077.8953718	total: 4.45s	remaining: 23s
162:	learn: 886358.3265003	total: 4.47s	remaining: 23s
163:	learn: 884512.3828822	total: 4.5s	remaining: 22.9s
164:	learn: 883135.8668530	total: 4.52s	remaining: 22.9s
165:	learn: 882087.3319278	total: 4.55s	remaining: 22.9s
166:	learn: 881405.5017756	total: 4.58s	remaining: 22.8s
167:	learn: 880892.2951188	total: 4.6s	remaining: 22.8s
168:	learn: 878593.5353479	total: 4.64s	remaining: 22.8s
169:	learn: 877238.6082620	total: 4.66s	remain

298:	learn: 760479.0188465	total: 8.29s	remaining: 19.4s
299:	learn: 760344.3955507	total: 8.31s	remaining: 19.4s
300:	learn: 759860.5573137	total: 8.34s	remaining: 19.4s
301:	learn: 759513.0506228	total: 8.37s	remaining: 19.3s
302:	learn: 758768.8252581	total: 8.4s	remaining: 19.3s
303:	learn: 758427.3312690	total: 8.43s	remaining: 19.3s
304:	learn: 757850.8981520	total: 8.46s	remaining: 19.3s
305:	learn: 757365.4480504	total: 8.49s	remaining: 19.3s
306:	learn: 756375.2707165	total: 8.51s	remaining: 19.2s
307:	learn: 754266.5937170	total: 8.54s	remaining: 19.2s
308:	learn: 753743.4980733	total: 8.56s	remaining: 19.2s
309:	learn: 753200.6797242	total: 8.59s	remaining: 19.1s
310:	learn: 752294.7893645	total: 8.62s	remaining: 19.1s
311:	learn: 751732.9906615	total: 8.64s	remaining: 19.1s
312:	learn: 749990.5478555	total: 8.67s	remaining: 19s
313:	learn: 749493.1026360	total: 8.7s	remaining: 19s
314:	learn: 747834.3182456	total: 8.72s	remaining: 19s
315:	learn: 746236.5354300	total: 8.75s

446:	learn: 666963.3551553	total: 12.3s	remaining: 15.2s
447:	learn: 665810.3612314	total: 12.3s	remaining: 15.2s
448:	learn: 665176.4192770	total: 12.4s	remaining: 15.2s
449:	learn: 664560.6169801	total: 12.4s	remaining: 15.1s
450:	learn: 664156.8129079	total: 12.4s	remaining: 15.1s
451:	learn: 663835.9147958	total: 12.4s	remaining: 15.1s
452:	learn: 663443.7185121	total: 12.5s	remaining: 15.1s
453:	learn: 662943.7715296	total: 12.5s	remaining: 15s
454:	learn: 662511.6708710	total: 12.5s	remaining: 15s
455:	learn: 662363.5395214	total: 12.5s	remaining: 15s
456:	learn: 661826.2130125	total: 12.6s	remaining: 14.9s
457:	learn: 661593.9040365	total: 12.6s	remaining: 14.9s
458:	learn: 660818.5125486	total: 12.6s	remaining: 14.9s
459:	learn: 660355.7071863	total: 12.7s	remaining: 14.9s
460:	learn: 659761.2234387	total: 12.7s	remaining: 14.8s
461:	learn: 659601.2599903	total: 12.7s	remaining: 14.8s
462:	learn: 659203.1165991	total: 12.7s	remaining: 14.8s
463:	learn: 658770.0692529	total: 12.

596:	learn: 616899.9120090	total: 16.5s	remaining: 11.2s
597:	learn: 616222.5857365	total: 16.5s	remaining: 11.1s
598:	learn: 615714.2041378	total: 16.6s	remaining: 11.1s
599:	learn: 615260.7858279	total: 16.6s	remaining: 11.1s
600:	learn: 614957.9745377	total: 16.6s	remaining: 11s
601:	learn: 614560.2801423	total: 16.7s	remaining: 11s
602:	learn: 614016.9152170	total: 16.7s	remaining: 11s
603:	learn: 613716.4883067	total: 16.7s	remaining: 11s
604:	learn: 613638.0416255	total: 16.7s	remaining: 10.9s
605:	learn: 613484.2323478	total: 16.8s	remaining: 10.9s
606:	learn: 613150.5392390	total: 16.8s	remaining: 10.9s
607:	learn: 612690.8242954	total: 16.8s	remaining: 10.8s
608:	learn: 611999.4126956	total: 16.9s	remaining: 10.8s
609:	learn: 611708.8528171	total: 16.9s	remaining: 10.8s
610:	learn: 611494.1136985	total: 16.9s	remaining: 10.8s
611:	learn: 611151.1544873	total: 16.9s	remaining: 10.7s
612:	learn: 610995.6801168	total: 17s	remaining: 10.7s
613:	learn: 610850.0538639	total: 17s	rem

744:	learn: 583605.6177386	total: 20.6s	remaining: 7.04s
745:	learn: 583467.7024317	total: 20.6s	remaining: 7.01s
746:	learn: 583388.7254179	total: 20.6s	remaining: 6.98s
747:	learn: 583035.6770192	total: 20.6s	remaining: 6.95s
748:	learn: 582900.9080031	total: 20.7s	remaining: 6.92s
749:	learn: 582832.4483948	total: 20.7s	remaining: 6.9s
750:	learn: 582654.7108821	total: 20.7s	remaining: 6.87s
751:	learn: 582305.1416267	total: 20.7s	remaining: 6.84s
752:	learn: 582198.4204413	total: 20.8s	remaining: 6.81s
753:	learn: 582014.4273965	total: 20.8s	remaining: 6.79s
754:	learn: 581924.7872898	total: 20.8s	remaining: 6.76s
755:	learn: 581719.7542870	total: 20.9s	remaining: 6.73s
756:	learn: 581091.8668081	total: 20.9s	remaining: 6.7s
757:	learn: 580745.3309456	total: 20.9s	remaining: 6.67s
758:	learn: 580683.7170721	total: 20.9s	remaining: 6.65s
759:	learn: 580259.6753846	total: 21s	remaining: 6.62s
760:	learn: 580070.5717195	total: 21s	remaining: 6.59s
761:	learn: 579788.5013203	total: 21s

894:	learn: 557660.4695148	total: 24.7s	remaining: 2.9s
895:	learn: 557476.8144798	total: 24.7s	remaining: 2.87s
896:	learn: 556672.3235020	total: 24.8s	remaining: 2.84s
897:	learn: 556556.8888260	total: 24.8s	remaining: 2.81s
898:	learn: 556406.6793600	total: 24.8s	remaining: 2.79s
899:	learn: 556176.3947814	total: 24.8s	remaining: 2.76s
900:	learn: 555978.8309746	total: 24.9s	remaining: 2.73s
901:	learn: 555957.2234465	total: 24.9s	remaining: 2.7s
902:	learn: 555867.8848048	total: 24.9s	remaining: 2.67s
903:	learn: 555746.2992102	total: 24.9s	remaining: 2.65s
904:	learn: 555618.9992885	total: 25s	remaining: 2.62s
905:	learn: 555389.2650170	total: 25s	remaining: 2.59s
906:	learn: 555383.6154280	total: 25s	remaining: 2.56s
907:	learn: 555086.8984474	total: 25s	remaining: 2.54s
908:	learn: 554768.1793163	total: 25.1s	remaining: 2.51s
909:	learn: 554586.5111676	total: 25.1s	remaining: 2.48s
910:	learn: 554487.7700625	total: 25.1s	remaining: 2.45s
911:	learn: 553877.2541864	total: 25.1s	r

In [195]:
model.best_score_

{'learn': {'MAE': 573855.8607048634}}

In [210]:
model.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'MAE',
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'],
 'iterations': 1000,
 'sampling_frequency': 'PerTree',
 'fold_permutation_block': 0,
 'leaf_estimation_method': 'Exact',
 'od_pval': 0,
 'counter_calc_method': 'SkipTest',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'ctr_leaf_count_limit': 18446744073709551615,
 'bayesian_matrix_reg': 0.10000000149011612,
 'one_hot_max_size': 2,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'od_type': 'Iter',
 'rsm': 1,
 'boost_from_average': True,
 'max_ctr_complexity': 4,
 'model_size_reg': 0.5,
 'simple_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Pr

In [196]:
# model.save_model("cb_trained")

In [216]:
y_train_test_pred = model.predict(X_train_test)

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.

In [215]:
relative_error(y_train_test, y_train_test_pred)

0.7061814875676478

In [ ]:
mean_absolute_error(y_train_test,y_train_test_pred )

Пока качество не айс. Ну чтош, сравню хотя бы как эта штука отрабатывает, и SimpleSolution в кегле на тестовых данных

### LightGBM

In [ ]:
preprocess_model.fit_transform(X_train_train)

In [ ]:
!pip install lightgbm

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
preprocess_model = make_pipeline(
    make_column_transformer(
        # convert date to months
        (DatePreprocessor(), ['date']),
        # fill nans for columns using their distributions
#         (DiscreteDistributionImputer(), ['build_tech', 'g_lift']),
        # fill left nans simply
        remainder='passthrough'
    ),
    # pandarize for other columntransform conversion, date, build_Tech and g_lift labels go first after first CT conversion
    pandarizer(X_train_train.columns, 'date'),
    # drop nans
#     NanDropper(),
    # drop kw columns that are highly disbalanced
#     make_column_transformer(
#         ('drop', ["kw{}".format(i) for i in range(1, 14) if i != 2]),
#         remainder='passthrough'
#     ),
    # pandarize again to check that everything is as must
#     pandarizer(without(X_labels, *["kw{}".format(i) for i in range(1, 14) if i != 2]), 
#                'date', 'build_tech', 'g_lift',
#                toint=['date', 'build_tech', 'g_lift', 'street_id', 'floor', 'area', 'rooms', 'balcon', 'n_photos', 'kw2']
#               ),
#     pandarizer(X_train_train.columns, 
#            'date',
#            toint=['date', 'build_tech', 'g_lift', 'street_id', 'floor', 'area', 'rooms', 'balcon', 'n_photos', 'kw2']
#           ),
    'passthrough'
)

In [ ]:
preprocess_model_normal = make_pipeline(
    make_column_transformer(
        # convert date to months
        (DatePreprocessor(), ['date']),
        # fill nans for columns using their distributions
        (DiscreteDistributionImputer(), ['build_tech', 'g_lift']),
        # fill left nans simply
        remainder=SimpleImputer()
    ),
    # pandarize for other columntransform conversion, date, build_Tech and g_lift labels go first after first CT conversion
    pandarizer(X_train_train.columns, 'date', 'build_tech', 'g_lift'),
    # drop kw columns that are highly disbalanced
#     make_column_transformer(
#         ('drop', ["kw{}".format(i) for i in range(1, 14) if i != 2]),
#         remainder='passthrough'
#     ),
    # pandarize again to check that everything is as must
#     pandarizer(without(X_labels, *["kw{}".format(i) for i in range(1, 14) if i != 2]), 
#                'date', 'build_tech', 'g_lift',
#                toint=['date', 'build_tech', 'g_lift', 'street_id', 'floor', 'area', 'rooms', 'balcon', 'n_photos', 'kw2']
#               ),
#     pandarizer(X_train_train.columns, 
#            'date',
#            toint=['date', 'build_tech', 'g_lift', 'street_id', 'floor', 'area', 'rooms', 'balcon', 'n_photos', 'kw2']
#           ),
    'passthrough'
)

In [ ]:
X_train_train.columns

In [ ]:
data_train_prep = preprocess_model.fit_transform(X_train_train).copy()

In [ ]:
data_train_prep = pd.DataFrame(np.hstack([data_train_prep, y_train_train[:, np.newaxis]]))
data_train_prep = data_train_prep.dropna()

In [ ]:
lgbm_model = LGBMRegressor()
lgbm_model.fit(data_train_prep.iloc[:, :-1], data_train_prep.iloc[:, -1])

In [ ]:
predicts = lgbm_model.predict(preprocess_model.fit_transform(X_train_test))

In [ ]:
relative_error(y_train_test, predicts)

In [ ]:
mean_absolute_error(y_train_test, predicts)

## Использование выбранной модели на тесте

In [203]:
%%time
data_test_preprocessed = create_preprocess_model_for_columns(data_test.columns).fit_transform(data_test)

['build_tech', 'g_lift']
datetime64[ns]
['build_tech', 'g_lift']
Wall time: 4min 13s


In [205]:
# data_test_preprocessed.to_csv("TestPreprocessed.csv", sep=",", index=False)

In [206]:
y_test_pred = model.predict(data_test_preprocessed)

In [207]:
answers = np.hstack((np.arange(100000, 200000)[:, np.newaxis], y_test_pred[:, np.newaxis]))

In [208]:
answers_pd = pd.DataFrame(answers, columns=["id", "price"])
answers_pd['id'] = answers_pd['id'].astype(np.int)
answers_pd.set_index('id')

,price
id,
100000,2.019673e+06
100001,2.425104e+06
100002,5.527786e+06
100003,1.349077e+06
100004,7.015918e+06
...,...
199995,1.516530e+07
199996,2.180920e+06
199997,2.614805e+06


In [209]:
answers_pd.to_csv("submission.csv", index=False)